In [2]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


In [4]:

trainfile = r'C:\Users\dipti\Documents\CIS508\Team Assignment 1\train.csv'
train_data = pd.read_csv(trainfile)

#train_data = pd.read_csv("C:/Users/admin/Downloads/Insurance Fraud - TRAIN-3000(1).csv")


testfile = r'C:\Users\dipti\Documents\CIS508\Team Assignment 1\test.csv'
test_data = pd.read_csv(testfile)

#test_data = pd.read_csv("C:/Users/admin/Downloads/Insurance Fraud -TEST-12900(1).csv")

print('Shape of train data')
print(train_data.shape)
print(test_data.shape)
print(train_data.head())    


Shape of train data
(1460, 81)
(1459, 80)
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCo

In [6]:
#To do encoding just for the catogerical variables
from sklearn import preprocessing


for f in train_data.columns:
    if train_data[f].dtype=='object':
        print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_data[f].values) + list(test_data[f].values))
        train_data[f] = lbl.transform(list(train_data[f].values))
        test_data[f] = lbl.transform(list(test_data[f].values))

### Traindata

In [8]:
## Here we will check the percentage of nan values present in each feature
## 1 -step make the list of features which has missing values
features_with_na=[features for features in train_data.columns if train_data[features].isnull().sum()>1]
## 2- step print the feature name and the percentage of missing values

for feature in features_with_na:
    print(feature, np.round(train_data[feature].isnull().mean(), 4),  ' % missing values')

LotFrontage 0.1774  % missing values
MasVnrArea 0.0055  % missing values
GarageYrBlt 0.0555  % missing values


In [10]:
## Replace missing value with a new label
features_nan=[feature for feature in train_data.columns if train_data[feature].isnull().sum()>1 and train_data[feature].dtypes=='O']

def replace_cat_feature(train_data,features_nan):
    data=train_data.copy()
    data[features_nan]=data[features_nan].fillna('Missing')
    return data

dataset=replace_cat_feature(train_data,features_nan)

dataset[features_nan].isnull().sum()

Series([], dtype: float64)

In [12]:
## Now lets check for numerical variables the contains missing values
numerical_with_nan=[feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtypes!='O']

## We will print the numerical nan variables and percentage of missing values

for feature in numerical_with_nan:
    print("{}: {}% missing value".format(feature,np.around(dataset[feature].isnull().mean(),4)))

LotFrontage: 0.1774% missing value
MasVnrArea: 0.0055% missing value
GarageYrBlt: 0.0555% missing value


In [14]:
## Replacing the numerical Missing Values

for feature in numerical_with_nan:
    ## We will replace by using median since there are outliers
    median_value=dataset[feature].median()
    
    ## create a new feature to capture nan values
    dataset[feature+'nan']=np.where(dataset[feature].isnull(),1,0)
    dataset[feature].fillna(median_value,inplace=True)
    
dataset[numerical_with_nan].isnull().sum()
    

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [18]:
feature_scale=[feature for feature in dataset.columns if feature not in ['Id','SalePrice']]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(dataset[feature_scale])

MinMaxScaler()

In [19]:
scaler.transform(dataset[feature_scale])

array([[0.23529412, 0.75      , 0.15068493, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.20205479, ..., 0.        , 0.        ,
        0.        ],
       [0.23529412, 0.75      , 0.1609589 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.29411765, 0.75      , 0.15410959, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.1609589 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.75      , 0.18493151, ..., 0.        , 0.        ,
        0.        ]])

In [21]:
#transform the train and test set, and add on the Id and SalePrice variables
data = pd.concat([dataset[['Id', 'SalePrice']].reset_index(drop=True),
                     pd.DataFrame(scaler.transform(dataset[feature_scale]), columns=feature_scale)],
                     axis=1)


In [22]:
dataset = dataset.drop(['LotFrontage','MasVnrArea','GarageYrBlt'], axis =1)

In [24]:
test_data.isnull().sum().sort_values(ascending = False).head(20)

LotFrontage     227
GarageYrBlt      78
MasVnrArea       15
BsmtFullBath      2
BsmtHalfBath      2
GarageCars        1
TotalBsmtSF       1
BsmtUnfSF         1
BsmtFinSF2        1
BsmtFinSF1        1
GarageArea        1
KitchenQual       0
Fireplaces        0
Functional        0
TotRmsAbvGrd      0
Id                0
KitchenAbvGr      0
BedroomAbvGr      0
HalfBath          0
FireplaceQu       0
dtype: int64

### Testdata

In [26]:
## Here we will check the percentage of nan values present in each feature
## 1 -step make the list of features which has missing values
features_with_na=[features for features in test_data.columns if test_data[features].isnull().sum()>1]
## 2- step print the feature name and the percentage of missing values

for feature in features_with_na:
    print(feature, np.round(test_data[feature].isnull().mean(), 4),  ' % missing values')

LotFrontage 0.1556  % missing values
MasVnrArea 0.0103  % missing values
BsmtFullBath 0.0014  % missing values
BsmtHalfBath 0.0014  % missing values
GarageYrBlt 0.0535  % missing values


In [28]:
## Replace missing value with a new label
features_nan=[feature for feature in test_data.columns if test_data[feature].isnull().sum()>1 and test_data[feature].dtypes=='O']

def replace_cat_feature(test_data,features_nan):
    testdata=test_data.copy()
    testdata[features_nan]=testdata[features_nan].fillna('Missing')
    return testdata

testdataset=replace_cat_feature(test_data,features_nan)

testdataset[features_nan].isnull().sum()

Series([], dtype: float64)

In [30]:
## Now lets check for numerical variables the contains missing values
numerical_with_nan=[feature for feature in testdataset.columns if testdataset[feature].isnull().sum()>0 and testdataset[feature].dtypes!='O']

## We will print the numerical nan variables and percentage of missing values

for feature in numerical_with_nan:
    print("{}: {}% missing value".format(feature,np.around(testdataset[feature].isnull().mean(),4)))

LotFrontage: 0.1556% missing value
MasVnrArea: 0.0103% missing value
BsmtFinSF1: 0.0007% missing value
BsmtFinSF2: 0.0007% missing value
BsmtUnfSF: 0.0007% missing value
TotalBsmtSF: 0.0007% missing value
BsmtFullBath: 0.0014% missing value
BsmtHalfBath: 0.0014% missing value
GarageYrBlt: 0.0535% missing value
GarageCars: 0.0007% missing value
GarageArea: 0.0007% missing value


In [32]:
## Replacing the numerical Missing Values

for feature in numerical_with_nan:
    ## We will replace by using median since there are outliers
    median_value=testdataset[feature].median()
    
    ## create a new feature to capture nan values
    testdataset[feature+'nan']=np.where(testdataset[feature].isnull(),1,0)
    testdataset[feature].fillna(median_value,inplace=True)
    
testdataset[numerical_with_nan].isnull().sum()
    

LotFrontage     0
MasVnrArea      0
BsmtFinSF1      0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
BsmtFullBath    0
BsmtHalfBath    0
GarageYrBlt     0
GarageCars      0
GarageArea      0
dtype: int64

In [34]:
for i in testdataset:
    if i not in dataset:
        print(i)

LotFrontage
MasVnrArea
GarageYrBlt
BsmtFinSF1nan
BsmtFinSF2nan
BsmtUnfSFnan
TotalBsmtSFnan
BsmtFullBathnan
BsmtHalfBathnan
GarageCarsnan
GarageAreanan


In [36]:
## Now lets check for numerical variables the contains missing values
numerical_with_nan=[feature for feature in testdataset.columns if testdataset[feature].isnull().sum()>1 and testdataset[feature].dtypes!='O']

## We will print the numerical nan variables and percentage of missing values

for feature in numerical_with_nan:
    print("{}: {}% missing value".format(feature,np.around(testdataset[feature].isnull().mean(),4)))

In [38]:
feature_scale=[feature for feature in testdataset.columns if feature not in ['Id','SalePrice']]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(testdataset[feature_scale])

MinMaxScaler()

In [41]:
scaler.transform(testdataset[feature_scale])
#transform the train and test set, and add on the Id and SalePrice variables
testdataset = pd.concat([testdataset[['Id']].reset_index(drop=True),
                         pd.DataFrame(scaler.transform(testdataset[feature_scale]), columns=feature_scale)],
                         axis=1)


In [43]:
testdataset

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MasVnrAreanan,BsmtFinSF1nan,BsmtFinSF2nan,BsmtUnfSFnan,TotalBsmtSFnan,BsmtFullBathnan,BsmtHalfBathnan,GarageYrBltnan,GarageCarsnan,GarageAreanan
0,1461,-0.117647,0.08,-0.115477,-0.026661,1.0,0.5,0.333333,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1462,-0.117647,0.12,-0.115446,-0.026660,1.0,0.5,0.000000,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1463,-0.116263,0.12,-0.115664,-0.026660,1.0,0.5,0.000000,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1464,-0.116263,0.12,-0.115539,-0.026661,1.0,0.5,0.000000,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1465,-0.114187,0.12,-0.116632,-0.026663,1.0,0.5,0.000000,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,-0.112803,0.16,-0.117318,-0.026664,1.0,0.5,0.333333,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1455,2916,-0.112803,0.16,-0.117318,-0.026664,1.0,0.5,0.333333,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1456,2917,-0.117647,0.12,-0.112980,-0.026658,1.0,0.5,0.333333,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,2918,-0.115398,0.12,-0.116039,-0.026661,1.0,0.5,0.333333,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
TrainCols = list(dataset.columns.values)
TestCols = list(testdataset.columns.values)
print(TrainCols)
print(TestCols)

['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal', 'MoSold

In [47]:
# Seperate Target column from Train Data
Xtrain = dataset.drop(labels= ['SalePrice'], axis = 1)
Ytrain = dataset[['SalePrice']].copy()
print("Train Set shape:")
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = testdataset.drop(['LotFrontage',
'MasVnrArea',
'GarageYrBlt',
'BsmtFullBathnan',
'BsmtHalfBathnan',
'BsmtFinSF1nan',
'BsmtFinSF2nan',
'BsmtUnfSFnan',
'TotalBsmtSFnan',
'GarageCarsnan',
'GarageAreanan'], axis =1)
print(Xtest.shape)# Seperate Target column from Train Data
Xtrain = dataset.drop(labels= ['SalePrice'], axis = 1)
Ytrain = dataset[['SalePrice']].copy()
print("Train Set shape:")
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = testdataset.drop(['LotFrontage',
'MasVnrArea',
'GarageYrBlt',
'BsmtFullBathnan',
'BsmtHalfBathnan',
'BsmtFinSF1nan',
'BsmtFinSF2nan',
'BsmtUnfSFnan',
'TotalBsmtSFnan',
'GarageCarsnan',
'GarageAreanan'], axis =1)
print(Xtest.shape)

Train Set shape:
(1460, 80)
(1460, 1)
(1459, 80)
Train Set shape:
(1460, 80)
(1460, 1)
(1459, 80)


In [49]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train, y_test = train_test_split(Xtrain, Ytrain, test_size=0.3, random_state=0) 

In [51]:
X_test1

,Id,MSSubClass,MSZoning,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
529,530,20,3,32668,1,2,0,3,0,1,...,4,4,0,3,2007,8,2,0,0,0
491,492,50,3,9490,1,2,3,3,0,4,...,2,4,0,8,2006,8,4,0,0,0
459,460,50,3,7015,1,2,0,0,0,0,...,4,4,0,7,2009,8,4,0,0,0
279,280,60,3,10005,1,2,3,3,0,4,...,4,4,0,3,2008,8,4,0,0,0
655,656,160,4,1680,1,2,3,3,0,4,...,4,4,0,3,2010,8,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,272,20,3,39104,1,2,0,2,0,1,...,4,4,0,4,2008,8,4,0,0,0
445,446,20,3,9855,1,2,3,3,0,0,...,2,4,0,11,2009,0,4,0,0,0
654,655,20,3,10437,1,2,0,3,0,4,...,4,4,0,8,2008,8,4,0,0,0
1280,1281,20,3,9808,1,2,0,3,0,4,...,4,4,0,3,2009,8,4,0,0,0


In [53]:
#Decision Tree Regressor ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

clf = DecisionTreeRegressor()
clf.fit(X_train1, y_train)
clf_predict_Train=clf.predict(X_train1)

#clf.feature_importances_
mean_squared_error(y_train,clf_predict_Train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,clf_predict_Train)))
clf_predict_Test=clf.predict(X_test1)
mean_squared_error(y_test,clf_predict_Test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,clf_predict_Test)))


RMSE (training) for Decision Tree:  0.000000
RMSE (Test Data) for Decision Tree:1311700178.655251


In [55]:
clf_predict_Test=clf.predict(Xtest)

In [57]:
submission=testdataset[['Id']]
submission['SalePrice']=clf_predict_Test
submission.to_csv("results2.1.csv", index = None)

In [59]:
#Random Forest Regressor==============================================================================
#=================================================================================================
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor()
rfc.fit(X_train1, y_train)
rfc_predict_Train=rfc.predict(X_train1)

mean_squared_error(y_train,rfc_predict_Train)
print("RMSE (training) for Random Forest:{0:10f}".format(mean_squared_error(y_train,rfc_predict_Train)))
rfc_predict_Test=rfc.predict(X_test1)
mean_squared_error(y_test,rfc_predict_Test)
print("RMSE (Test Data) for Random Forest:{0:10f}".format(mean_squared_error(y_test,rfc_predict_Test)))



RMSE (training) for Random Forest:137509390.183133
RMSE (Test Data) for Random Forest:889008068.001869


In [62]:
rfc_predict_test = rfc.predict(Xtest)

In [64]:
submission=testdataset[['Id']]
submission['SalePrice']=rfc_predict_test
submission.to_csv("results2.2.csv", index = None)

In [67]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train1,y_train)
lr_predict_train=lr.predict(X_train1)

mean_squared_error(y_train, lr_predict_train)
print("RMSE (training) for linear regressor:{0:10f}".format(mean_squared_error(y_train,lr_predict_train)))
lr_predict_test = lr.predict(X_test1)
mean_squared_error(y_test,lr_predict_test)
print("RMSE (Test) for all linear regressor:{}".format(mean_squared_error(y_test,lr_predict_test)))

RMSE (training) for linear regressor:722984729.811340
RMSE (Test) for all linear regressor:2271200438.749049


In [69]:
lr_predict_test1 = lr.predict(Xtest)

In [71]:
submission=testdataset[['Id']]
submission['SalePrice']=lr_predict_test1
submission.to_csv("results2.3.csv", index = None)

In [73]:
from sklearn.neural_network import MLPRegressor
mlp=MLPRegressor()
mlp.fit(X_train1,y_train)
mlp_predict_train = mlp.predict(X_train1)

mean_squared_error(y_train,mlp_predict_train)
print("RMSE (Train) for MLP :{}",mean_squared_error(y_train,mlp_predict_train))
mlp_predict_test = mlp.predict(X_test1)
mean_squared_error(y_test,mlp_predict_test)
print("RMSE (Test) for MLP:{}",mean_squared_error(y_test,mlp_predict_test))

RMSE (Train) for MLP :{} 2315145937.6756916
RMSE (Test) for MLP:{} 3074035851.607442


In [75]:
mlp_predict_test1 = mlp.predict(Xtest)
submission=testdataset[['Id']]
submission['SalePrice']=mlp_predict_test1
submission.to_csv("results2.5.csv", index = None)

In [77]:
from sklearn.svm import SVR
svr=SVR()
svr.fit(X_train1,y_train)
svr_predict_train = svr.predict(X_train1)

mean_squared_error(y_train,svr_predict_train)
print("RMSE (Train) for SVR :{}",mean_squared_error(y_train,svr_predict_train))
svr_predict_test = svr.predict(X_test1)
mean_squared_error(y_test,svr_predict_test)
print("RMSE (Test) for SVR:{}",mean_squared_error(y_test,svr_predict_test))

RMSE (Train) for SVR :{} 6407310408.975444
RMSE (Test) for SVR:{} 7136965028.321759


### Gradient Descent Booster

In [79]:
from sklearn.linear_model import SGDRegressor
sgd=SGDRegressor()
sgd.fit(X_train1,y_train)
sgd_predict_train = sgd.predict(X_train1)

mean_squared_error(y_train,sgd_predict_train)
print("RMSE (Train) for SGD :{}",mean_squared_error(y_train,sgd_predict_train))
sgd_predict_test = sgd.predict(X_test1)
mean_squared_error(y_test,sgd_predict_test)
print("RMSE (Test) for SGD:{}",mean_squared_error(y_test,sgd_predict_test))

RMSE (Train) for SGD :{} 4.550189819103496e+32
RMSE (Test) for SGD:{} 4.501248293815367e+32


In [81]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [  RandomForestRegressor(), DecisionTreeRegressor(), MLPRegressor(), LinearRegression() ]
      
S_Train, S_Test = stacking(models,                   
                           X_train1, y_train, X_test1,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
                                        
                           n_folds=4, 
                                                    
                           verbose=2, shuffle=True)



___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [4]

model  0:     [RandomForestRegressor]
    fold  0:  [19044.33875000]
    fold  1:  [16986.77003906]
    fold  2:  [18344.78290196]
    fold  3:  [16495.81427451]
    ----
    MEAN:     [17717.92649138] + [1022.30945262]
    FULL:     [17718.50893346]

model  1:     [DecisionTreeRegressor]
    fold  0:  [27548.60937500]
    fold  1:  [28833.03125000]
    fold  2:  [29791.63529412]
    fold  3:  [30058.03529412]
    ----
    MEAN:     [29057.82780331] + [983.25508046]
    FULL:     [29056.13111546]

model  2:     [MLPRegressor]
    fold  0:  [42586.75260922]
    fold  1:  [36354.17973529]
    fold  2:  [41429.56272864]
    fold  3:  [35357.06753469]
    ----
    MEAN:     [38931.89065196] + [31

In [83]:
#STACKING - CONTRUCT A Random Forest MODEL==============================
model = RandomForestRegressor()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)

In [85]:
mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
mean_squared_error(y_test,y_pred_test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))

RMSE (training) for Decision Tree:111271885.999746
RMSE (Test Data) for Decision Tree:702761671.312184


In [87]:
#STACKING - CONTRUCT A Random Forest MODEL==============================
model = DecisionTreeRegressor()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)

mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
mean_squared_error(y_test,y_pred_test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))


RMSE (training) for Decision Tree:  0.000000
RMSE (Test Data) for Decision Tree:983051834.515982


In [89]:
#STACKING - CONTRUCT A Random Forest MODEL==============================
model = MLPRegressor()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)

mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
mean_squared_error(y_test,y_pred_test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))


RMSE (training) for Decision Tree:736169835.450675
RMSE (Test Data) for Decision Tree:1059305295.096684


In [91]:
#STACKING - CONTRUCT A Random Forest MODEL==============================
model = LinearRegression()
    
model = model.fit(S_Train, y_train)
y_pred_train = model.predict(S_Train)
y_pred_test = model.predict(S_Test)

mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))
mean_squared_error(y_test,y_pred_test)
print("RMSE (Test Data) for Decision Tree:{0:10f}".format(mean_squared_error(y_test,y_pred_test)))


RMSE (training) for Decision Tree:775034090.656805
RMSE (Test Data) for Decision Tree:1035793995.733554


In [93]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [  RandomForestRegressor(), DecisionTreeRegressor(), MLPRegressor(), LinearRegression() ]
      
Q_Train, Q_Test = stacking(models,                   
                           X_train1, y_train, Xtest,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
                                        
                           n_folds=4, 
                                                    
                           verbose=2, shuffle=True)



___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [4]

model  0:     [RandomForestRegressor]
    fold  0:  [19146.92101563]
    fold  1:  [17397.70707031]
    fold  2:  [18622.10556863]
    fold  3:  [16948.64713725]
    ----
    MEAN:     [18028.84519795] + [889.83349397]
    FULL:     [18029.32165362]

model  1:     [DecisionTreeRegressor]
    fold  0:  [27174.24609375]
    fold  1:  [29670.89062500]
    fold  2:  [28375.60392157]
    fold  3:  [30375.63137255]
    ----
    MEAN:     [28899.09300322] + [1227.29120473]
    FULL:     [28898.16046967]

model  2:     [MLPRegressor]
    fold  0:  [42138.97300666]
    fold  1:  [36944.33329407]
    fold  2:  [40084.72400148]
    fold  3:  [35508.19717548]
    ----
    MEAN:     [38669.05686942] + [25

In [96]:
#STACKING - CONTRUCT A Random Forest MODEL==============================
model1 = RandomForestRegressor()
    
model1 = model1.fit(Q_Train, y_train)
y_pred_train = model1.predict(Q_Train)
y_pred_test = model1.predict(Q_Test)

mean_squared_error(y_train,y_pred_train)
print("RMSE (training) for Decision Tree:{0:10f}".format(mean_squared_error(y_train,y_pred_train)))


RMSE (training) for Decision Tree:121966600.531236


In [97]:
submission=testdataset[['Id']]
submission['SalePrice']=y_pred_test
submission.to_csv("results2.6.csv", index = None)

In [98]:
model2 = DecisionTreeRegressor()

model2 = model2.fit(Q_Train,y_train)
y_predict_train = model2.predict(Q_Train)
Y_predict_test = model2.predict(Q_Test)
mean_squared_error(y_train,y_predict_train)
print('RMSE (train) for Decision Tree Regressor:{}'.format(mean_squared_error(y_train,y_predict_train)))

RMSE (train) for Decision Tree Regressor:0.0


In [99]:
submission=testdataset[['Id']]
submission['SalePrice']=Y_predict_test
submission.to_csv("results2.7.csv", index = None)

In [100]:
model3 = MLPRegressor()

model3 = model3.fit(Q_Train, y_train)
y_predict_train3 = model3.predict(Q_Train)
y_predict_test3 = model3.predict(Q_Test)
mean_squared_error(y_train,y_predict_train3)
print("RMSE (Train) for MLP Regressor{}".format(mean_squared_error(y_train,y_predict_train3)))


RMSE (Train) for MLP Regressor796578308.4989439


In [101]:
submissions= testdataset['Id']
submissions['SalePrice']=y_predict_test3
submissions.to_csv("results2.8.csv", index = None)
submissions

0                                                         1461
1                                                         1462
2                                                         1463
3                                                         1464
4                                                         1465
                                   ...                        
1455                                                      2916
1456                                                      2917
1457                                                      2918
1458                                                      2919
SalePrice    [775194.2794546046, 775694.9777252866, 775691....
Name: Id, Length: 1460, dtype: object

In [102]:
model4 = MLPRegressor()

model4 = model4.fit(Q_Train, y_train)
y_predict_train4 = model4.predict(Q_Train)
y_predict_test4 = model4.predict(Q_Test)
mean_squared_error(y_train,y_predict_train4)
print("RMSE (Train) for MLP Regressor{}".format(mean_squared_error(y_train,y_predict_train4)))

RMSE (Train) for MLP Regressor813988971.1270338


In [103]:
submission=testdataset[['Id']]
submission['SalePrice']=y_predict_test4
submission.to_csv("results2.10.csv", index = None)